# Color Detection

In [1]:
import numpy as np 
import cv2

def get_limits(color):
    
    c = np.uint8([[color]]) # bgr values which you want to convert to hsv
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    

    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    
    lowerLimit = np.array(lowerLimit, dtype = np.uint8)
    upperLimit = np.array(upperLimit, dtype = np.uint8)
    
    return lowerLimit, upperLimit

## Color Detection using Pillow

In [26]:
import cv2
from PIL import Image

cap = cv2.VideoCapture(0)

yellow = [0 ,255, 255] # yellow in bgr color space
blue = [255, 0, 0]

# open webcam
while True:
    ret, frame = cap.read() # reads the image from webcam
    
    # color detection stuff happens has to happen before the image is shown
    
    # first convert imaage from bgr to hsv color space
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # use get_color function to return lower and upper limit in hsv cs for inRange function 
    lowerLimit, upperLimit = get_limits(blue)
    
    # generates a mask(T/F array) of all the pixel values that contain that color
    mask = cv2.inRange(img_hsv, lowerLimit, upperLimit) # cv2.inRange(source, lower_limit, upper_limit)
    
    # converts the nmpy opencv represntation to a pillow object
    mask_ = Image.fromarray(mask) 
    bbox = mask_.getbbox() # pillow has a function called getbbox that automatically get the bounding box for a set of pixels
    
    # prints location of bounding box of each yellow pixel
    # print(bbox) 
    
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))
    
    cv2.imshow('webcam', mask) # shows image on a new frame, to see mask, change frame to mask
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [27]:
img_hsv[:,:,1]

array([[ 20,  20,  18, ..., 130, 173, 186],
       [ 20,  20,  19, ..., 144, 126, 152],
       [ 20,  20,  19, ..., 137, 136, 125],
       ...,
       [  6,   8,  10, ...,  80,  77,  79],
       [  9,  11,  11, ...,  78,  75,  75],
       [ 12,  14,  13, ...,  76,  75,  75]], dtype=uint8)

HSV color space works like a cylinder. We want to define a margin of error between each h s and v value so that the program can pick up an oobject of a general color.

## Color Detection using Saturation thresholding

In [4]:
import cv2
from PIL import Image

cap = cv2.VideoCapture(0)

yellow = [0 ,255, 255] # yellow in bgr color space
blue = [255, 0, 0]

# open webcam
while True:
    ret, frame = cap.read() # reads the image from webcam
    
    # color detection stuff happens has to happen before the image is shown
    
    # first convert imaage from bgr to hsv color space
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # use get_color function to return lower and upper limit in hsv cs for inRange function 
    lowerLimit, upperLimit = get_limits(yellow)
    
    # generates a mask(T/F array) of all the pixel values that contain that color
    mask = cv2.inRange(img_hsv, lowerLimit, upperLimit) # cv2.inRange(source, lower_limit, upper_limit)
    
    # Apply saturation thresholding 
    sat_thresh = 40 # adjust this value based on your environment
    _, sat_mask = cv2.threshold(img_hsv[:,:,1], 170, 255, cv2.THRESH_BINARY) # img_hsv[:, :, 1] is the satruation channel of the img_hsv tensor
    # adjust second parameter of threshold based on enrionment, or replace with sat_thresh and change that variable 
    
    mask = cv2.bitwise_and(mask, sat_mask)
    
    
    # converts the nmpy opencv represntation to a pillow object
    mask_ = Image.fromarray(mask) 
    bbox = mask_.getbbox() # pillow has a function called getbbox that automatically get the bounding box for a set of pixels
    
    # prints location of bounding box of each yellow pixel
    # print(bbox) 
    
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 10)
    
    cv2.imshow('webcam', mask) # shows image on a new frame, to see mask, change frame to mask
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

### What is img_hsv[:, :, 1]?

In OpenCV, the HSV color space consists of three components: Hue, Saturation, and Value. The `img_hsv[:, :, 1]` refers to the Saturation channel of the image in HSV color space.

Here's a breakdown of what each channel represents:

- Hue (img_hsv[:, :, 0]): It represents the color itself, and it ranges from 0 to 179 in OpenCV.
- Saturation (img_hsv[:, :, 1]): It represents the intensity of the color. A value of 0 means grayscale (no color), and a value of 255 means fully saturated color.
- Value (img_hsv[:, :, 2]): It represents the brightness of the color. A higher value means brighter pixels.

In the code you provided, `img_hsv[:, :, 1]` is extracting the Saturation channel of the image. This channel is then thresholded using the `cv2.threshold` function to create a binary mask (`sat_mask`). Pixels with saturation values below the specified threshold (`sat_thresh`) will be set to 0 (black) in the mask, and others will be set to 255 (white). This helps filter out pixels with low saturation, which might be caused by excessive light in the environment.

## Using Countours to find multiple objects of same color

In [9]:
import cv2
from PIL import Image

cap = cv2.VideoCapture(0)

yellow = [0 ,255, 255] # yellow in bgr color space
blue = [255, 0, 0]

# open webcam
while True:
    ret, frame = cap.read() # reads the image from webcam
    
    # color detection stuff happens has to happen before the image is shown
    
    # first convert imaage from bgr to hsv color space
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # use get_color function to return lower and upper limit in hsv cs for inRange function 
    lowerLimit, upperLimit = get_limits(yellow)
    
    # generates a mask(T/F array) of all the pixel values that contain that color
    mask = cv2.inRange(img_hsv, lowerLimit, upperLimit) # cv2.inRange(source, lower_limit, upper_limit)
    
    # Apply saturation thresholding 
    sat_thresh = 180 # adjust this value based on your environment
    _, sat_mask = cv2.threshold(img_hsv[:,:,1], sat_thresh, 255, cv2.THRESH_BINARY) # img_hsv[:, :, 1] is the satruation channel of the img_hsv tensor
    # adjust second parameter of threshold based on enrionment, or replace with sat_thresh and change that variable 
    
    mask = cv2.bitwise_and(mask, sat_mask)
    
    # find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    # Draw bounding boxes around each contour
    
    for cnt in contours:
        
        if cv2.contourArea(cnt) > 2000:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    """
    for cnt in contours:
        print(cv2.contourArea(cnt))
        if cv2.contourArea(cnt) > 0:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    """

    
    # converts the nmpy opencv represntation to a pillow object
    mask_ = Image.fromarray(mask) 
    #bbox = mask_.getbbox() # pillow has a function called getbbox that automatically get the bounding box for a set of pixels
    
    # prints location of bounding box of each yellow pixel
    # print(bbox) 
    
    
    """
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 10)
    """
    
    cv2.imshow('webcam', frame) # shows image on a new frame, to see mask, change frame to mask
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In this modified code, after applying the color and saturation thresholds, we use the cv2.findContours function to detect contours in the binary mask. Then, for each contour, we compute its bounding box using cv2.boundingRect and draw a rectangle around it on the original frame. This allows you to detect and draw bounding boxes around multiple yellow objects separately.

## Summary

Because I was in the library, the lighting caued for a high saturation value. Saturation is the amount of gray that is in a color. Gray is effectively a mixture of black and white. So in cases where there is high lighitng, some colors will have more white(the parts of the image where lighting is well lit) and some parts of the image are more dark(absense of the light hitting it ie shadows). This is my understanding of it. I used 2 masks, one that extracts the color, and the other being a thresholding mask, where if saturation value is above 100 it goes to 255, and if it is below 100 it goes to 0. This sat mask effectively ignores spaces/colors have high lights.